In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter,BravyiKitaevSuperFastMapper
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

In [7]:
def BeH2_adaptvqe(k,dist):
    
    # Define Molecule
    molecule = MoleculeInfo(
        ["Be","H","H"],[(0.0,0.0,0.0),(dist, 0.0, 0.0),(-1*dist, 0.0, 0.0)],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    driver = PySCFDriver().from_molecule(molecule)

    problem = driver.run()
    converter = QubitConverter(BravyiKitaevSuperFastMapper())
    transformer = ActiveSpaceTransformer(
    num_electrons=4, #how many electrons we have in our active space
    num_spatial_orbitals=3, #how many orbitals we have in our active space
    )
    
    #problem = transformer.transform(problem=problem)
    print('粒子数{0}，空间轨道数 {1}'.format(problem.num_particles,problem.num_spatial_orbitals))
    
    initial_state = HartreeFock(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter)
    ansatz = UCCSD(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter,initial_state=initial_state)
    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
    
    adapt_vqe = AdaptVQE(vqe,max_iterations=30,threshold=1e-4)
    adapt_vqe.supports_aux_operators = lambda: True  # temporary fix
    solver = GroundStateEigensolver(converter, adapt_vqe)
    adapt_result = solver.solve(problem)
    print('AdaptVQE 已经完成：{0}%'.format(((k+1)/80)*100))
    return adapt_result

In [8]:
def BeH2_uccsd(dist,k):
    # Define Molecule
    molecule = MoleculeInfo(
        ["Be","H","H"],[(0.0,0.0,0.0),(dist, 0.0, 0.0),(-1*dist, 0.0, 0.0)],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    driver = PySCFDriver().from_molecule(molecule)

    problem = driver.run()
    converter = QubitConverter(JordanWignerMapper())
    transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_spatial_orbitals=3, #how many orbitals we have in our active space
    )
    problem = transformer.transform(problem=problem)
    print('粒子数{0}，空间轨道数 {1}'.format(problem.num_particles,problem.num_spatial_orbitals))
    
    initial_state = HartreeFock(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter)
    ansatz = UCCSD(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter,initial_state=initial_state)
    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
    
    # adapt_vqe = AdaptVQE(vqe)
    # adapt_vqe.supports_aux_operators = lambda: True  # temporary fix
    solver = GroundStateEigensolver(converter, vqe)
    uccsd_result = solver.solve(problem)
    print('UCCSD ansatz 共有 {0} 个参数'.format(ansatz.num_parameters))
    print('UCCSD_VQE 已经完成：{0}%'.format(((k+1)/80)*100))
    
    return uccsd_result

In [6]:
distance = np.linspace(0.5,2.5,80)
adapt_vqe_result = []
uccsd_vqe_result = []

for index,dis in enumerate(distance):
    adapt_vqe_result.append(BeH2_adaptvqe(dist=dis,k=index))
    uccsd_vqe_result.append(BeH2_uccsd(dist=dis,k=index))

粒子数(3, 3)，空间轨道数 7


TypeError: object of type 'bool' has no len()

In [9]:
result1 = BeH2_uccsd(dist=1.7,k=1)

粒子数(1, 1)，空间轨道数 3
UCCSD ansatz 共有 8 个参数
UCCSD_VQE 已经完成：2.5%


In [33]:
k = result1.raw_result.optimal_circuit
tmp =k.decompose().to_instruction()

In [27]:
tmp.decompose().to_instruction()

Instruction(name='EvolvedOps', num_qubits=6, num_clbits=0, params=[ParameterVectorElement(t[0]), ParameterVectorElement(t[1]), ParameterVectorElement(t[2]), ParameterVectorElement(t[3]), ParameterVectorElement(t[4]), ParameterVectorElement(t[5]), ParameterVectorElement(t[6]), ParameterVectorElement(t[7])])

In [52]:
from qiskit.circuit import QuantumCircuit
circuit = QuantumCircuit(6)
circuit.append(tmp,range(6))
t = circuit.decompose().decompose()
circuit.decompose(reps=1).draw()

┌───┐┌───────────────────────────────────────┐»
q_0: ┤ X ├┤0                                      ├»
     └───┘│                                       │»
q_1: ─────┤1                                      ├»
          │                                       │»
q_2: ─────┤2                                      ├»
     ┌───┐│  exp(-it (IIIIXY + IIIIYX))(1.0*t[0]) │»
q_3: ┤ X ├┤3                                      ├»
     └───┘│                                       │»
q_4: ─────┤4                                      ├»
          │                                       │»
q_5: ─────┤5                                      ├»
          └───────────────────────────────────────┘»
«     ┌───────────────────────────────────────┐»
«q_0: ┤0                                      ├»
«     │                                       │»
«q_1: ┤1                                      ├»
«     │                                       │»
«q_2: ┤2                                      ├»
«     │  exp(-it (IIIXZY + IIIYZX))(1.0*t[1]) │»
«q_3: ┤3                                      ├»
«     │                                       │»
«q_4: ┤4                                      ├»
«     │                                       │»
«q_5: ┤5                                      ├»
«     └───────────────────────────────────────┘»
«     ┌───────────────────────────────────────┐»
«q_0: ┤0                                      ├»
«     │                                       │»
«q_1: ┤1                                      ├»
«     │                                       │»
«q_2: ┤2                                      ├»
«     │  exp(-it (IXYIII + IYXIII))(1.0*t[2]) │»
«q_3: ┤3                                      ├»
«     │                                       │»
«q_4: ┤4                                      ├»
«     │                                       │»
«q_5: ┤5                                      ├»
«     └───────────────────────────────────────┘»
«     ┌───────────────────────────────────────┐»
«q_0: ┤0                                      ├»
«     │                                       │»
«q_1: ┤1                                      ├»
«     │                                       │»
«q_2: ┤2                                      ├»
«     │  exp(-it (XZYIII + YZXIII))(1.0*t[3]) │»
«q_3: ┤3                                      ├»
«     │                                       │»
«q_4: ┤4                                      ├»
«     │                                       │»
«q_5: ┤5                                      ├»
«     └───────────────────────────────────────┘»
«     ┌─────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                            ├»
«     │                                                                                             │»
«q_1: ┤1                                                                                            ├»
«     │                                                                                             │»
«q_2: ┤2                                                                                            ├»
«     │  exp(-it (IYYIXY + IXYIYY + IXXIXY + IYXIYY + IXYIXX + IYYIYX + IYXIXX + IXXIYX))(1.0*t[4]) │»
«q_3: ┤3                                                                                            ├»
«     │                                                                                             │»
«q_4: ┤4                                                                                            ├»
«     │                                                                                             │»
«q_5: ┤5                                                                                            ├»
«     └─────────────────────────────────────────────────────────────────────────────────────────────┘»
«     ┌───────────────────────────────────────────────────────────

In [44]:
type(t)

qiskit.circuit.instruction.Instruction

In [55]:
t.draw()

┌───────────┐┌───────────────────────┐┌───────────────────────┐»
q_0: ┤ U3(π,0,π) ├┤0                      ├┤0                      ├»
     └───────────┘│                       ││                       │»
q_1: ─────────────┤1                      ├┤1                      ├»
                  │                       ││                       │»
q_2: ─────────────┤2                      ├┤2                      ├»
     ┌───────────┐│  exp(it IIIIXY)(t[0]) ││  exp(it IIIIYX)(t[0]) │»
q_3: ┤ U3(π,0,π) ├┤3                      ├┤3                      ├»
     └───────────┘│                       ││                       │»
q_4: ─────────────┤4                      ├┤4                      ├»
                  │                       ││                       │»
q_5: ─────────────┤5                      ├┤5                      ├»
                  └───────────────────────┘└───────────────────────┘»
«     ┌───────────────────────┐┌───────────────────────┐»
«q_0: ┤0                      ├┤0                      ├»
«     │                       ││                       │»
«q_1: ┤1                      ├┤1                      ├»
«     │                       ││                       │»
«q_2: ┤2                      ├┤2                      ├»
«     │  exp(it IIIXZY)(t[1]) ││  exp(it IIIYZX)(t[1]) │»
«q_3: ┤3                      ├┤3                      ├»
«     │                       ││                       │»
«q_4: ┤4                      ├┤4                      ├»
«     │                       ││                       │»
«q_5: ┤5                      ├┤5                      ├»
«     └───────────────────────┘└───────────────────────┘»
«     ┌───────────────────────┐┌───────────────────────┐»
«q_0: ┤0                      ├┤0                      ├»
«     │                       ││                       │»
«q_1: ┤1                      ├┤1                      ├»
«     │                       ││                       │»
«q_2: ┤2                      ├┤2                      ├»
«     │  exp(it IXYIII)(t[2]) ││  exp(it IYXIII)(t[2]) │»
«q_3: ┤3                      ├┤3                      ├»
«     │                       ││                       │»
«q_4: ┤4                      ├┤4                      ├»
«     │                       ││                       │»
«q_5: ┤5                      ├┤5                      ├»
«     └───────────────────────┘└───────────────────────┘»
«     ┌───────────────────────┐┌───────────────────────┐»
«q_0: ┤0                      ├┤0                      ├»
«     │                       ││                       │»
«q_1: ┤1                      ├┤1                      ├»
«     │                       ││                       │»
«q_2: ┤2                      ├┤2                      ├»
«     │  exp(it XZYIII)(t[3]) ││  exp(it YZXIII)(t[3]) │»
«q_3: ┤3                      ├┤3                      ├»
«     │                       ││                       │»
«q_4: ┤4                      ├┤4                      ├»
«     │                       ││                       │»
«q_5: ┤5                      ├┤5                      ├»
«     └───────────────────────┘└───────────────────────┘»
«     ┌───────────────────────┐┌───────────────────────┐»
«q_0: ┤0                      ├┤0                      ├»
«     │                       ││                       │»
«q_1: ┤1                      ├┤1                      ├»
«     │                       ││                       │»
«q_2: ┤2                      ├┤2                      ├»
«     │  exp(it IYYIXY)(t[4]) ││  exp(it IXYIYY)(t[4]) │»
«q_3: ┤3                      ├┤3                      ├»
«     │                       ││                       │»
«q_4: ┤4                      ├┤4                      ├»
«     │                       ││                       │»
«q_5: ┤5                      ├┤5                      ├»
«     └───────────────────────┘└───────────────────────┘»
«     ┌───────────────────────┐┌───────────────────────┐»
«q_0: ┤0             